In [12]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import pickle

# Load dataset
df = pd.read_csv("asl_dataset.csv")

# Separate features (landmarks) and labels
X = df.drop(columns=['label']).values
y = df['label'].values

# Normalize landmark data (Scale between 0 and 1)
scaler = MinMaxScaler()
X = scaler.fit_transform(X)  # Apply normalization

# Save the scaler for real-time prediction
with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

print("Scaler saved successfully!")

# Encode labels (convert letters to numbers)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Save label encoder for real-time usage
with open("label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)

# Split dataset (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Convert labels to categorical (one-hot encoding for softmax)
y_train = keras.utils.to_categorical(y_train, num_classes=len(label_encoder.classes_))
y_test = keras.utils.to_categorical(y_test, num_classes=len(label_encoder.classes_))

# Define a deeper neural network model with reduced dropout
model = keras.Sequential([
    keras.layers.Input(shape=(42,)),  # 21 hand landmarks (x, y)
    keras.layers.Dense(512, activation="relu"),  # Increased neurons
    keras.layers.Dropout(0.1),  # Reduced dropout
    keras.layers.Dense(256, activation="relu"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(len(label_encoder.classes_), activation="softmax")  # Output layer
])

# Use a lower learning rate
optimizer = keras.optimizers.Adam(learning_rate=0.0005)  # Reduced from default 0.001

# Compile model
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=16, validation_data=(X_test, y_test))

# Save the trained model
model.save("asl_mlp_model.h5")

print("MLP Model trained and saved successfully!")

# Load trained model
model = keras.models.load_model("asl_mlp_model.h5")

# Evaluate on test data
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Scaler saved successfully!
Epoch 1/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.0741 - loss: 3.2407 - val_accuracy: 0.1190 - val_loss: 3.1777
Epoch 2/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1542 - loss: 3.1180 - val_accuracy: 0.1905 - val_loss: 2.9869
Epoch 3/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1735 - loss: 2.8883 - val_accuracy: 0.2083 - val_loss: 2.6674
Epoch 4/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2443 - loss: 2.5019 - val_accuracy: 0.2679 - val_loss: 2.2887
Epoch 5/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3248 - loss: 2.1494 - val_accuracy: 0.3869 - val_loss: 1.9464
Epoch 6/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4066 - loss: 1.8669 - val_accuracy: 0.3810 - val_loss: 1.7637
Epoch 7/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4442 - loss: 1.6607 - val_accuracy: 0.3929 - val_loss: 1.7430
Epoch 8/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4316 - loss: 1.5429

MLP Model trained and saved successfully!
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8943 - loss: 0.5295  
Test Accuracy: 88.10%
